In [5]:
import os
import pandas as pd
import ast
from gbfs.services import SystemDiscoveryService

def process_gbfs_data():
    if not os.path.exists('rawdata'):
        os.makedirs('rawdata')

    ds = SystemDiscoveryService()

    lyft_system_id = [x.get('System ID') for x in ds.systems if 'lyft' in x.get('Name').lower()][0]

    lyft_client = ds.instantiate_client(lyft_system_id)

    lyft_feed_names = lyft_client.feed_names

    specified_feeds = ['station_information', 'vehicle_types', 'system_pricing_plans', 'free_bike_status', 'station_status']

    dfs = []

    for feed_name in lyft_feed_names:
        try:
            feed_data = lyft_client.request_feed(feed_name)
            if feed_name in specified_feeds:
                df = pd.DataFrame(feed_data)
                df['System ID'] = lyft_system_id
                df['Feed Name'] = feed_name
                dfs.append(df)
        except RuntimeError as e:
            print(f"Feed '{feed_name}' is not available: {e}")

    init_data = pd.concat(dfs, ignore_index=True)

    for feed_name in specified_feeds:
        feed_data = init_data[init_data['Feed Name'] == feed_name]['data']
        df = pd.DataFrame(feed_data.iloc[0])
        if 'rental_uris' in df.columns:
            df.drop(columns=['rental_uris'], inplace=True)
        file_path = os.path.join('rawdata', f"{feed_name}.csv")

        df.fillna(0, inplace=True)
        
        df.to_csv(file_path, index=False)

    print("Data saved successfully in rawdata directory.")

    # Load and process station_status CSV
    station_status_file = 'rawdata/station_status.csv'
    if os.path.exists(station_status_file):
        df = pd.read_csv(station_status_file)
        df['vehicle_types_available'] = df['vehicle_types_available'].apply(ast.literal_eval)
        for index, row in df.iterrows():
            for item in row['vehicle_types_available']:
                vehicle_type_id = item['vehicle_type_id']
                count = item['count']
                df.at[index, f'vehicle_type_id_{vehicle_type_id}_count'] = count
        df.drop(columns=['vehicle_types_available'], inplace=True)
        
        df.fillna(0, inplace=True)

        df.to_csv(station_status_file, index=False)
        print("Processed station_status.csv successfully.")
    else:
        print(f"Error: {station_status_file} not found.")

    # Load, process, and save system_pricing_plans CSV
    system_pricing_plans_file = 'rawdata/system_pricing_plans.csv'
    if os.path.exists(system_pricing_plans_file):
        df = pd.read_csv(system_pricing_plans_file)
        df['per_min_pricing'] = df['per_min_pricing'].apply(ast.literal_eval)
        df['per_min_rate'] = df['per_min_pricing'].apply(lambda x: x[0]['rate'])
        df['per_min_start'] = df['per_min_pricing'].apply(lambda x: x[0]['start'])
        df['per_min_interval'] = df['per_min_pricing'].apply(lambda x: x[0]['interval'])

        df['description'] = df.apply(lambda row: f"${row['price']} unlock fee, ${row['per_min_rate']} per minute.", axis=1)

        modified_data = df[['is_taxable', 'currency', 'name', 'price', 'description', 'plan_id', 'per_min_rate', 'per_min_start', 'per_min_interval']]

        df.drop(columns=['per_min_pricing'], inplace=True)
        
        df.fillna(0, inplace=True)

        df.to_csv(system_pricing_plans_file, index=False)
        print("Processed system_pricing_plans.csv successfully.")
    else:
        print(f"Error: {system_pricing_plans_file} not found.")

process_gbfs_data()


Data saved successfully in rawdata directory.
Processed station_status.csv successfully.
Processed system_pricing_plans.csv successfully.
